In [89]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 20000)

We load the source data. We have to load the "Old" cleaned data table, because we cant left join geo_data and Updated_Temperature directly on deveui id (deveui id is hot encoded on Updated_Temperature). So we first join the ID to geo_data and then we will join geo_data and Updated_Temperature on id.

In [90]:
df_old = pd.read_csv('../data/preprocessed/cleaned_data.csv', delimiter=",")
df = pd.read_csv('../data/preprocessed/Updated_Temperature.csv', delimiter=",")
df_geo = pd.read_csv('../data/preprocessed/geo_data.csv', delimiter=",")

In [91]:
# rename, so we can left join with df_old and get all the ids
df_geo.rename(columns={'Sensor':'deveui'}, inplace=True) 

# add increment id from 1-77 so wa have a semantic container id afterwards
df_geo = df_geo.reset_index()
df_geo['container_id'] = df_geo.index + 1

In [92]:
# merge the cleaned_data.csv and geo_data.csv, so we have geo data for each ID
tmp = pd.merge(df_old,df_geo,on='deveui',how='left')

# Drop unwanted cols

In [93]:
tmp.drop('deveui', 1, inplace=True)
tmp.drop('decoded_data.sensor_data.Events', 1, inplace=True)
tmp.drop('decoded_data.sensor_data.Height 1', 1, inplace=True)
tmp.drop('decoded_data.sensor_data.Temperature', 1, inplace=True)
tmp.drop('decoded_data.sensor_data.Tilt', 1, inplace=True)
tmp.drop('decoded_data.sensor_data.Tx Event', 1, inplace=True)
tmp.drop('Sens. Typ', 1, inplace=True)
tmp.drop('Location', 1, inplace=True)
tmp.drop('Cont. Typ', 1, inplace=True)
tmp.drop('Address', 1, inplace=True)
tmp.drop('loc', 1, inplace=True)
tmp.drop('point', 1, inplace=True)
tmp.drop('altitude', 1, inplace=True)

# Mergen

In [94]:
# left join the geo_data and updated_temperatre.csv
tmp2 = pd.merge(df,tmp,on='id',how='left')

ist_industriegebiet und ist_wohngebiet als 0 und 1 mappen

In [95]:
tmp2["ist_industriegebiet"] = tmp2["ist_industriegebiet"].astype(int)
tmp2["ist_wohngebiet"] = tmp2["ist_wohngebiet"].astype(int)

# Generate more time fields

In [96]:
tmp2['stunde'] = pd.DatetimeIndex(tmp2['created_at']).hour
tmp2['minute'] = pd.DatetimeIndex(tmp2['created_at']).minute
tmp2['sekunde'] = pd.DatetimeIndex(tmp2['created_at']).second

# Drop Columns

In [97]:
tmp2.drop('unix_time', 1, inplace=True)
tmp2.drop('index', 1, inplace=True)
tmp2.drop('decoded_data.sensor_data.Events', 1, inplace=True)
tmp2.drop('parsed_time_unix', 1, inplace=True)

# Rename columns

In [98]:
tmp2.rename(columns={'decoded_data.sensor_data.Height 1':'height_in_cm'}, inplace=True)
tmp2.rename(columns={'decoded_data.sensor_data.Temperature':'temperatur_in_grad'}, inplace=True) 
tmp2.rename(columns={'decoded_data.sensor_data.Tilt':'tilt_in_grad'}, inplace=True) 
tmp2.rename(columns={'decoded_data.sensor_data.Tx Event':'messungszähler'}, inplace=True) 
tmp2.rename(columns={'is_holiday':'ist_feiertag'}, inplace=True) 
tmp2.rename(columns={'rain (mm)':'niederschlag_in_mm'}, inplace=True) 
tmp2.rename(columns={'Hight Delta':'height_delta'}, inplace=True) 
tmp2.rename(columns={'Increase':'height_increase'}, inplace=True) 
tmp2.rename(columns={'parsed_time_month':'monat'}, inplace=True) 
tmp2.rename(columns={'parsed_time_year':'jahr'}, inplace=True) 
tmp2.rename(columns={'parsed_time_week':'kalenderwoche'}, inplace=True) 
tmp2.rename(columns={'parsed_time_weekday':'wochentag'}, inplace=True) 
tmp2.rename(columns={'created_at':'datum'}, inplace=True) 
tmp2.rename(columns={'Farbe':'farbe'}, inplace=True) 
tmp2.rename(columns={'lat':'laengengrad'}, inplace=True) 
tmp2.rename(columns={'lon':'breitengrad'}, inplace=True)

In [99]:
tmp2

,id,height_in_cm,temperatur_in_grad,tilt_in_grad,messungszähler,ist_feiertag,niederschlag_in_mm,height_delta,height_increase,monat,jahr,kalenderwoche,wochentag,DEVEUI_70B3D500700016DA,DEVEUI_70B3D500700016DE,DEVEUI_70B3D500700016DF,DEVEUI_70B3D500700016E0,DEVEUI_70B3D500700016E5,DEVEUI_70B3D500700016E6,DEVEUI_70B3D500700016E7,DEVEUI_70B3D500700016EB,DEVEUI_70B3D500700016EE,DEVEUI_70B3D500700016F1,DEVEUI_70B3D500700016F2,DEVEUI_70B3D500700016F4,DEVEUI_70B3D500700016F6,DEVEUI_70B3D500700016F7,DEVEUI_70B3D500700016FA,DEVEUI_70B3D500700016FC,DEVEUI_70B3D50070001700,DEVEUI_70B3D50070001701,DEVEUI_70B3D50070001704,DEVEUI_70B3D50070001706,DEVEUI_70B3D50070001708,DEVEUI_70B3D50070001709,DEVEUI_70B3D5007000170F,DEVEUI_70B3D50070001710,DEVEUI_70B3D50070001712,DEVEUI_70B3D50070001713,DEVEUI_70B3D50070001714,DEVEUI_70B3D50070001715,DEVEUI_70B3D50070001716,DEVEUI_70B3D5007000171A,DEVEUI_70B3D5007000171E,DEVEUI_70B3D5007000171F,DEVEUI_70B3D50070001722,DEVEUI_70B3D50070001724,DEVEUI_70B3D50070001725,DEVEUI_70B3D50070001726,DEVEUI_70B3D50070001727,DEVEUI_70B3D5007000172B,DEVEUI_70B3D5007000172C,DEVEUI_70B3D5007000172D,DEVEUI_70B3D5007000172E,DEVEUI_70B3D50070001730,DEVEUI_70B3D50070001731,DEVEUI_70B3D50070001733,DEVEUI_70B3D50070001734,DEVEUI_70B3D50070001736,DEVEUI_70B3D50070001737,DEVEUI_70B3D50070001738,DEVEUI_70B3D50070001739,DEVEUI_70B3D5007000173A,DEVEUI_70B3D5007000173C,DEVEUI_70B3D5007000173E,DEVEUI_70B3D50070001740,DEVEUI_70B3D50070001742,DEVEUI_70B3D50070001743,DEVEUI_70B3D50070001747,DEVEUI_70B3D5007000174D,DEVEUI_70B3D5007000174F,DEVEUI_70B3D50070001750,DEVEUI_70B3D50070001759,DEVEUI_70B3D5007000175A,DEVEUI_70B3D5007000175E,DEVEUI_70B3D50070001764,DEVEUI_70B3D50070001766,DEVEUI_70B3D50070001770,DEVEUI_70B3D50070001772,DEVEUI_70B3D50070001774,DEVEUI_70B3D50070001777,DEVEUI_70B3D50070001779,DEVEUI_70B3D5007000177C,DEVEUI_70B3D50070001780,DEVEUI_70B3D50070001781,DEVEUI_70B3D50070001782,DEVEUI_70B3D50070001786,DEVEUI_70B3D50070001787,DEVEUI_70B3D50070001788,DEVEUI_70B3D50070001789,datum,farbe,laengengrad,breitengrad,ist_industriegebiet,ist_wohngebiet,container_id,stunde,minute,sekunde
0,4110576,112,14.0,1,193,0,0.0,0.0,0.000000e+00,5,2021,18,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021-05-08 20:26:00.596017,white,50.134640,8.710140,0,1,1,20,26,0
1,4110093,112,17.0,1,192,0,0.0,4.0,1.109070e-06,5,2021,18,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021-05-08 19:26:05.713448,white,50.134640,8.710140,0,1,1,19,26,5
2,4109607,108,15.0,0,191,0,0.0,-18.0,-5.000290e-06,5,2021,18,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021-05-08 18:25:59.323267,white,50.134640,8.710140,0,1,1,18,25,59
3,4109118,126,19.0,1,190,0,0.0,8.0,2.222260e-06,5,2021,18,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021-05-08 17:25:59.523137,white,50.134640,8.710140,0,1,1,17,25,59
4,4108627,118,20.0,0,189,0,0.0,-4.0,-2.777940e-07,5,2021,18,5,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021-05-08 16:25:59.577446,white,50.134640,8.710140,0,1,1,16,25,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
515032,598759,104,19.0,5,142,0,0.0,0.0,0.000000e+00,5,2020,19,5,0,0,0,0,0,

# Rearrange Column order

In [100]:
tmp2 = tmp2[['id',
 'container_id',
 'messungszähler',
 'height_in_cm',
 'temperatur_in_grad',
 'tilt_in_grad',
 'niederschlag_in_mm',
 'height_delta',
 'height_increase',
 'farbe',
 'laengengrad',
 'breitengrad',
 'ist_industriegebiet',
 'ist_wohngebiet',
 'ist_feiertag',
 'datum',
 'sekunde',
 'minute',
 'stunde',
 'wochentag',
 'kalenderwoche',
 'monat',
 'jahr',
 'DEVEUI_70B3D500700016DA',
 'DEVEUI_70B3D500700016DE',
 'DEVEUI_70B3D500700016DF',
 'DEVEUI_70B3D500700016E0',
 'DEVEUI_70B3D500700016E5',
 'DEVEUI_70B3D500700016E6',
 'DEVEUI_70B3D500700016E7',
 'DEVEUI_70B3D500700016EB',
 'DEVEUI_70B3D500700016EE',
 'DEVEUI_70B3D500700016F1',
 'DEVEUI_70B3D500700016F2',
 'DEVEUI_70B3D500700016F4',
 'DEVEUI_70B3D500700016F6',
 'DEVEUI_70B3D500700016F7',
 'DEVEUI_70B3D500700016FA',
 'DEVEUI_70B3D500700016FC',
 'DEVEUI_70B3D50070001700',
 'DEVEUI_70B3D50070001701',
 'DEVEUI_70B3D50070001704',
 'DEVEUI_70B3D50070001706',
 'DEVEUI_70B3D50070001708',
 'DEVEUI_70B3D50070001709',
 'DEVEUI_70B3D5007000170F',
 'DEVEUI_70B3D50070001710',
 'DEVEUI_70B3D50070001712',
 'DEVEUI_70B3D50070001713',
 'DEVEUI_70B3D50070001714',
 'DEVEUI_70B3D50070001715',
 'DEVEUI_70B3D50070001716',
 'DEVEUI_70B3D5007000171A',
 'DEVEUI_70B3D5007000171E',
 'DEVEUI_70B3D5007000171F',
 'DEVEUI_70B3D50070001722',
 'DEVEUI_70B3D50070001724',
 'DEVEUI_70B3D50070001725',
 'DEVEUI_70B3D50070001726',
 'DEVEUI_70B3D50070001727',
 'DEVEUI_70B3D5007000172B',
 'DEVEUI_70B3D5007000172C',
 'DEVEUI_70B3D5007000172D',
 'DEVEUI_70B3D5007000172E',
 'DEVEUI_70B3D50070001730',
 'DEVEUI_70B3D50070001731',
 'DEVEUI_70B3D50070001733',
 'DEVEUI_70B3D50070001734',
 'DEVEUI_70B3D50070001736',
 'DEVEUI_70B3D50070001737',
 'DEVEUI_70B3D50070001738',
 'DEVEUI_70B3D50070001739',
 'DEVEUI_70B3D5007000173A',
 'DEVEUI_70B3D5007000173C',
 'DEVEUI_70B3D5007000173E',
 'DEVEUI_70B3D50070001740',
 'DEVEUI_70B3D50070001742',
 'DEVEUI_70B3D50070001743',
 'DEVEUI_70B3D50070001747',
 'DEVEUI_70B3D5007000174D',
 'DEVEUI_70B3D5007000174F',
 'DEVEUI_70B3D50070001750',
 'DEVEUI_70B3D50070001759',
 'DEVEUI_70B3D5007000175A',
 'DEVEUI_70B3D5007000175E',
 'DEVEUI_70B3D50070001764',
 'DEVEUI_70B3D50070001766',
 'DEVEUI_70B3D50070001770',
 'DEVEUI_70B3D50070001772',
 'DEVEUI_70B3D50070001774',
 'DEVEUI_70B3D50070001777',
 'DEVEUI_70B3D50070001779',
 'DEVEUI_70B3D5007000177C',
 'DEVEUI_70B3D50070001780',
 'DEVEUI_70B3D50070001781',
 'DEVEUI_70B3D50070001782',
 'DEVEUI_70B3D50070001786',
 'DEVEUI_70B3D50070001787',
 'DEVEUI_70B3D50070001788',
 'DEVEUI_70B3D50070001789']]

# Export Data

In [101]:
tmp2.to_csv('../data/preprocessed/cleaned_data_v2.csv', index=False)

In [104]:
tmp2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 515037 entries, 0 to 515036
Data columns (total 100 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   id                       515037 non-null  int64  
 1   container_id             515037 non-null  int64  
 2   messungszähler           515037 non-null  int64  
 3   height_in_cm             515037 non-null  int64  
 4   temperatur_in_grad       515037 non-null  float64
 5   tilt_in_grad             515037 non-null  int64  
 6   niederschlag_in_mm       515037 non-null  float64
 7   height_delta             514863 non-null  float64
 8   height_increase          514863 non-null  float64
 9   farbe                    515037 non-null  object 
 10  laengengrad              515037 non-null  float64
 11  breitengrad              515037 non-null  float64
 12  ist_industriegebiet      515037 non-null  int32  
 13  ist_wohngebiet           515037 non-null  int32  
 14  ist